In [1]:
from modules import chatbot
# from modules import device, model, context_input, response_input, embs_gen, cand_emb_gen, score, get_cand_embs, response_table, chatbot

/home/jake/anaconda3/envs/chatbot/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a m

In [2]:
# cand_embs = get_cand_embs()
# response_table;

In [11]:
# def chatbot(query):
#     query = [query]
#     poly_embs = embs_gen(*context_input(query))
#     score_lst = score(poly_embs, cand_embs)
#     idx = score_lst.argmax(1)
#     idx = int(idx[0])
#     return response_table['response'][idx][0]

In [5]:
query = '우울한 날들이 계속되고 있어.'
answer = chatbot(query)
print(answer)

자꾸 우울한 생각이 드시는 거 같네요. 좋은 날이 올 거라 믿어요.


In [2]:
import pickle

with open('./data/train_data_source.pickle', 'rb') as f:
    train = pickle.load(f)
with open('./data/val_data_source.pickle', 'rb') as f:
    dev = pickle.load(f)

In [3]:
import pandas as pd

data = {
    'context' : [],
    'response': []
}

for sample in train:
    data['context'].append(sample['context'])
    data['response'].append([sample['responses'][0]])

df = pd.DataFrame(data)

In [4]:
with open('response_data.pickle', 'wb') as f:
    pickle.dump(df, f)